In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
cargos = spark.read.load("historial_cargos.csv", delimiter = ",", inferSchema=True, header = True, format = "csv")

In [4]:
cargos.show(10)

+--------------+-----------------+--------------+-----+--------------+-------------+-------------+--------------+----------------------------------+
|fecha_de_cargo|      descripcion|     categoria|monto| forma_de_pago|realizado_por|  comentarios|limite_de_pago|deuda_a_plazos_recurrente_o_normal|
+--------------+-----------------+--------------+-----+--------------+-------------+-------------+--------------+----------------------------------+
|    2024-05-10|      Flores Mamá| Regalos Otros|200.0|        Costco|        Aylin|Dia de madres|    2024-06-03|                            Normal|
|    2024-05-09|      Flores Mamá| Regalos Otros|200.0|        Costco|        Aylin|Dia de madres|    2024-06-03|                            Normal|
|    2024-05-17|Credito de nomina|Credito Nomina|500.0|Crédito Nomina|        David|         NULL|    2024-05-17|                            Plazos|
|    2024-06-02|Credito de nomina|Credito Nomina|500.0|Crédito Nomina|        David|         NULL|    2024

In [5]:
cargos.createOrReplaceTempView("cargos")

In [6]:
ingresos = spark.read.load("ingresos_recurrentes_detalle.csv", delimiter = ",", inferSchema=True, header = True, format = "csv")

In [7]:
from pyspark.sql import functions as F

In [8]:
ingresos = ingresos.filter(F.col("categoria") == "Salario")

In [9]:
ingresos.createOrReplaceTempView("ingresos")

In [10]:
spark.sql("""
        select 
            i.fecha_de_ingreso AS lower_salary_date,
            i2.fecha_de_ingreso AS upper_salary_date
          FROM ingresos AS i
          LEFT JOIN ingresos AS i2
            ON i.indice = i2.indice - 1
        
          """).createOrReplaceTempView("wage_date_range")

In [16]:
spark.sql("""
        SELECT "tengo que borrar las tablas originales y recrearlas"
        
          """).show(30)

+---------------------------------------------------+
|tengo que borrar las tablas originales y recrearlas|
+---------------------------------------------------+
|                               tengo que borrar ...|
+---------------------------------------------------+



In [12]:
spark.sql("""
        
        SELECT 
          * 
        FROM cargos AS c
        LEFT JOIN wage_date_range AS dates
          ON c.limite_de_pago > dates.lower_salary_date
          AND c.limite_de_pago <= dates.upper_salary_date
""").show()


+--------------+-------------------+--------------+-----------------+--------------+-------------+-------------+--------------+----------------------------------+-----------------+-----------------+
|fecha_de_cargo|        descripcion|     categoria|            monto| forma_de_pago|realizado_por|  comentarios|limite_de_pago|deuda_a_plazos_recurrente_o_normal|lower_salary_date|upper_salary_date|
+--------------+-------------------+--------------+-----------------+--------------+-------------+-------------+--------------+----------------------------------+-----------------+-----------------+
|    2024-02-29|Celular Samsung A54|  Electrónicos|555.5555555555555|         LikeU|        Aylin|         NULL|    2024-03-31|                            Plazos|       2024-03-29|       2024-04-14|
|    2024-03-29|Celular Samsung A54|  Electrónicos|555.5555555555555|         LikeU|        Aylin|         NULL|    2024-05-01|                            Plazos|       2024-04-29|       2024-05-14|
|    

In [13]:
spark.sql("""
        SELECT 
            realizado_por, forma_de_pago, limite_de_pago, lower_salary_date,
            SUM(MONTO) AS Monto_Total
        FROM gastos_ingresos
        GROUP BY
          realizado_por, forma_de_pago, limite_de_pago, lower_salary_date
          """).show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `gastos_ingresos` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 5 pos 13;
'Aggregate ['realizado_por, 'forma_de_pago, 'limite_de_pago, 'lower_salary_date], ['realizado_por, 'forma_de_pago, 'limite_de_pago, 'lower_salary_date, 'SUM('MONTO) AS Monto_Total#267]
+- 'UnresolvedRelation [gastos_ingresos], [], false
